# Zadaca 12

## Zadatak 1

<img src="1.jpeg" />

<img src="2.jpeg" />

## Zadatak 2

D0:

0  INF INF INF -1  INF  
1  INF INF 2  INF INF  
INF 2  0  INF INF -8   
-4  INF INF 0  3  INF  
INF 7  INF INF 0  INF  
INF 5  10  INF INF 0   


D1:

0  INF INF INF -1  INF  
1  INF INF 2  0  INF  
INF 2  0  INF INF -8   
-4  INF INF 0  -5  INF  
INF 7  INF INF 0  INF  
INF 5  10  INF INF 0   


D2:

0  INF INF INF -1  INF  
1  INF INF 2  0  INF  
3  2  0  4  2  -8   
-4  INF INF 0  -5  INF  
8  7  INF 9  0  INF  
6  5  10  7  5  0   


D3:

0  INF INF INF -1  INF  
1  INF INF 2  0  INF  
3  2  0  4  2  -8   
-4  INF INF 0  -5  INF  
8  7  INF 9  0  INF  
6  5  10  7  5  0   


D4:

0  INF INF INF -1  INF  
-2  INF INF 2  -3  INF  
0  2  0  4  -1  -8   
-4  INF INF 0  -5  INF  
5  7  INF 9  0  INF  
3  5  10  7  2  0   


D5:

0  6  INF 8  -1  INF  
-2  4  INF 2  -3  INF  
0  2  0  4  -1  -8   
-4  2  INF 0  -5  INF  
5  7  INF 9  0  INF  
3  5  10  7  2  0   


## Zadatak 3

In [ ]:
# a)
from queue import PriorityQueue
INF = 1e9
def dijkstra(graph, start, end, parent, prob):
    prob[start] = 1 # vjerojatnost da ce se moci doci do starta je maksimalna jer do njega nema cesta
    pq = PriorityQueue()
    # -1, jer trebam extraxt-max umjesto extract-min operaciju
    pq.put((-1, start))
    while(not pq.empty()):
        curr = pq.get()[1]
        for edge in graph.edges.whereStart(curr):
            if(prob[edge.end] < prob[curr] * edge.prob):
                prob[edge.end] = prob[curr] * edge.prob
                pq.put((-prob[edge.end], edge.end))
        
            

def shortestPath(graph, start, end):
    for edge in graph.edges:
        if edge.prob == 1:
            graph.removeEdge(edge) # makni ove s vjerojatnoscu 1 jer sigurno kamion nece moci proci tamo
    parent = [None] * graph.vertices
    prob =  [0] * graph.vertices
    dijkstra(graph, start, end, parent, prob)
    path = []
    curr = end
    while(curr != None):
        path.append(curr)
        curr = parent[curr]
    path.reverse()
    return path

In [ ]:
# b)
# 1)
def heaviestTruckInit(graph, start, end, weight):
    dp = [-1 for i in range(graph.size)]
    dp[0] = INF
    heaviestTruck(graph, start, end, dp)
    return dp

def heaviestTruck(graph, start, end, dp):
    max_weight = -1
    prev = None
    for edge in graph.edges.whereEnd(end):
        if(dp[edge.start][end] == -1):
            dp[edge.start] = heaviestTruck(graph, start, edge.start, dp)
        max_weight = max(max_weight, min(dp[edge.start], edge.weight))
    dp[end] = max_weight
    return dp[end]

# objasnjenje:
# Do svakog vrha v iz V moze se doci sa maksimalnom tezinom kamiona w. Ta maksimalna tezina je ili jednaka tezini najtezeg vrha u iz kojeg se moze doci do v ili tezini ceste izmedu tih vrhova, ovisno o tome koja je tezina manja.
# Npr, ako od osijeka do Belog manastira mogu otputovati kamionom tezine 30 tona, ali onda cesta do madarske granice podrzava kamion od maksimalno od 25 tona, najveca moguca tezina kamiona je 25 tona jer jedino on moze prijeci cijeli put

In [ ]:
# 2)
# Posto su tezine sigurno pozitivne (ceste ne mogu imati negativnu duzinu), koristimo Modificirani Dijsktra algoritam
def dijkstra(graph, start, end, truckWeight):
    dp = heaviestTruckInit(graph, start, end, truckWeight)
    for vertex in graph.vertices:
        if(dp[vertex] < truckWeight):
            for edge in graph.edges.whereStart(vertex):
                graph.removeEdge(edge)
    dist = [INF for i in range(graph.size)]
    dist[start] = 0
    pq = PriorityQueue()
    pq.put((0, start))
    while(not pq.empty()):
        curr = pq.get()[1]
        for edge in graph.edges.whereStart(curr):
            if(dist[edge.end] > dist[curr] + edge.weight):
                dist[edge.end] = dist[curr] + edge.weight
                pq.put((dist[edge.end], edge.end))
    return dist[end]

In [ ]:
# c)
# vertexi predstavljaju gradove
# Istocno bi znacilo da od odredista moze ici samo prema istoku, tj. da ne moze 2x posjetiti isti vertex (jer bi u suprotnom mogao ici i prema zapadu)
# takoder, ne smije ici juzno/sjeverno, sto znaci da ne smije 2x posjetiti vertex na istom levelu

# IDEJA: augmentirati graf tako da svakom vertexu u grafu nadodam vrijednost levela BFS algoritmom. Zatim, ako je level odredista veci od levela polazista, onda moze doci do odredista jer ide istocno, a ako je manji ili jednak, onda ne moze doci do odredista jer bi morao ici prema zapadu/sjeveru/jugu

from queue import Queue
def modified_bfs(graph, start, end):
    level = [-1 for i in range(graph.size)]
    level[start] = 0
    q = Queue()
    q.put(start)
    while(not q.empty()):
        curr = q.get()
        for edge in graph.edges.whereStart(curr):
            if(level[edge.end] == -1):
                level[edge.end] = level[curr] + 1
                q.put(edge.end)

def dijkstra(graph, start, end, truckWeight):
    dp = heaviestTruckInit(graph, start, end, truckWeight)
    # O(V)
    for vertex in graph.vertices:
        if(dp[vertex] < truckWeight):
            for edge in graph.edges.whereStart(vertex):
                graph.removeEdge(edge)
    # O(V + E)
    modified_bfs(graph, start, end) # ovo augmentira graf s levelima

    # O(E * lg V)
    dist = [INF for i in range(graph.size)]
    dist[start] = 0
    pq = PriorityQueue()
    pq.put((0, start))
    while(not pq.empty()):
        curr = pq.get()[1]
        for edge in graph.edges.whereStart(curr):
            if(dist[edge.end] > dist[curr] + edge.weight):
                dist[edge.end] = dist[curr] + edge.weight
                pq.put((dist[edge.end], edge.end))
    return dist[end]

# Zadatak 4

<img src="3.jpeg" />

<img src="4.jpeg" />

<img src="5.jpeg" />